<a href="https://colab.research.google.com/github/wyxuan02/GenerativeAI_class2025/blob/main/AI08_RAG01%E6%89%93%E9%80%A0%E5%90%91%E9%87%8F%E8%B3%87%E6%96%99%E5%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

課程範例：https://github.com/yenlung/AI-Demo/blob/master/%E3%80%90Demo06a%E3%80%91RAG01_%E6%89%93%E9%80%A0%E5%90%91%E9%87%8F%E8%B3%87%E6%96%99%E5%BA%AB.ipynb

### 1. 建立資料夾

In [ ]:
import os
upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)
print(f"請將你的 .txt, .pdf, .docx 檔案放到這個資料夾中： {upload_dir}")

### 2. 更新必要套件並引入

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx sentence-transformers faiss-cpu

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

### 3. 依 e5 建議加入

自訂支援 E5 的 embedding 模型（加上 "passage:" / "query:" 前綴）

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 4. 載入文件

In [ ]:
folder_path = upload_dir
documents = []
for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)
    if file.endswith(".txt"):
        loader = TextLoader(path)
    elif file.endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif file.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(path)
    else:
        continue
    documents.extend(loader.load())

### 5. 建立向量資料庫

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
vectorstore = FAISS.from_documents(split_docs, embedding_model)

### 6. 儲存向量資料庫

In [ ]:
vectorstore.save_local("healthymeal_faiss_db")

In [ ]:
!zip -r healthymeal_faiss_db.zip healthymeal_faiss_db

In [ ]:
print("✅ 壓縮好的向量資料庫已儲存為 'healthymeal_faiss_db.zip'，請下載此檔案備份。")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import nbformat

# 設定 notebook 檔案路徑（換成你找到的完整路徑）
notebook_path = "/content/drive/MyDrive/Colab Notebooks/AI08_RAG01打造向量資料庫.ipynb"

# 讀取 notebook
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# 移除所有輸出與 widgets metadata
for cell in nb.cells:
    if "outputs" in cell:
        cell["outputs"] = []
    if "execution_count" in cell:
        cell["execution_count"] = None

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# 儲存回 notebook
with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ 已移除 widgets metadata 與輸出，可以安全 push 到 GitHub！")


✅ 已移除 widgets metadata 與輸出，可以安全 push 到 GitHub！
